## Part 1: Building up a basic predictive model

### Model Building 

In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('cleaned_copy.csv')
data.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,glipizide,glyburide,pioglitazone,rosiglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,Caucasian,1,8,1,22,7,0.461538,MC,Orthopedics-Reconstructive,58,...,0,0,1,0,No,1,No,1,1,False
1,Caucasian,1,7,2,1,1,0.153846,MC,Nephrology,59,...,0,0,0,0,No,1,No,0,1,False
2,Caucasian,1,10,1,1,7,0.230769,MC,Emergency/Trauma,56,...,0,0,0,0,No,0,No,0,1,False
3,Caucasian,1,8,1,2,7,0.692308,MC,InternalMedicine,68,...,0,0,0,0,No,1,No,1,1,False
4,Caucasian,1,8,1,3,5,0.846154,UN,InternalMedicine,77,...,0,0,0,0,No,1,No,0,1,False


### Logestic Regression Model 

In [241]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [242]:
# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [243]:
# Define the features you want to select
selected_features = ['number_emergency', 'number_inpatient', 'number_diagnoses', 'num_medications', 'time_in_hospital']
# Feature Selection
X = data[selected_features]  # Select only the specified features
y = data['readmitted']
model = LogisticRegression(max_iter=10000)  # Increase max_iter
model.fit(X, y)
# Print the selected features
print("Selected Features:")
print(selected_features)

Selected Features:
['number_emergency', 'number_inpatient', 'number_diagnoses', 'num_medications', 'time_in_hospital']


In [244]:
# Train-test split
# Splitting the data into training and test sets for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

In [245]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [246]:
# Evaluate the model using cross-validation
# Using 10-fold cross-validation to evaluate the model's performance considering the imbalanced nature of the data
cv = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')

In [247]:
# Print cross-validation scores
print("Cross-validation AUC-ROC scores:", cv_scores)
print("Mean AUC-ROC score:", cv_scores.mean())

Cross-validation AUC-ROC scores: [0.59688389 0.60176208 0.59861291 0.55197156 0.63691423 0.60501924
 0.56312351 0.60943461 0.59255111 0.56978481]
Mean AUC-ROC score: 0.5926057938599864


In [248]:
# Evaluate the model on test set
# Making predictions on the test set and generating a classification report
from sklearn.metrics import classification_report, roc_auc_score
y_pred = model.predict(X_test)
print("Classification Report on Test Set:")
print(classification_report(y_test, y_pred))

Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.91      1.00      0.95      3447
        True       0.00      0.00      0.00       333

    accuracy                           0.91      3780
   macro avg       0.46      0.50      0.48      3780
weighted avg       0.83      0.91      0.87      3780



C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\bhuva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

In [249]:
# Calculate class imbalance ratio
class_counts = data['readmitted'].value_counts()
imbalance_ratio = class_counts[1] / class_counts[0]
print("Class Imbalance Ratio:", imbalance_ratio)

Class Imbalance Ratio: 0.10115371168861438


#### Oversampling

In [250]:
from imblearn.over_sampling import RandomOverSampler
# Balance the data using random oversampling
random_oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)

In [251]:
# Train the model
model.fit(X_train_resampled, y_train_resampled)

LogisticRegression(max_iter=10000)

In [252]:
# Evaluate the model using cross-validation
cv_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=10, scoring='roc_auc')

In [253]:
# Print cross-validation scores
print("Cross-validation AUC-ROC scores:", cv_scores)
print("Mean AUC-ROC score:", cv_scores.mean())

Cross-validation AUC-ROC scores: [0.59385001 0.59344092 0.60070218 0.5992596  0.58496224 0.59810278
 0.58904595 0.57878339 0.58988247 0.58524959]
Mean AUC-ROC score: 0.5913279128469143


In [254]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
print("Classification Report on Test Set:")
print(classification_report(y_test, y_pred))

Classification Report on Test Set:
              precision    recall  f1-score   support

       False       0.93      0.63      0.75      3447
        True       0.12      0.52      0.19       333

    accuracy                           0.62      3780
   macro avg       0.53      0.58      0.47      3780
weighted avg       0.86      0.62      0.70      3780



In [255]:
# Evaluate the AUC-ROC score on test set
auc_roc = roc_auc_score(y_test, y_pred)
print("AUC-ROC score on Test Set:", auc_roc)

AUC-ROC score on Test Set: 0.5758826711829322


In [256]:
import plotly.graph_objs as go
import plotly.express as px

# Calculate the counts of unique classes after oversampling
class_counts_resampled = y_train_resampled.value_counts()

# Create a bar plot
fig = go.Figure(go.Bar(
    x=class_counts_resampled.index,
    y=class_counts_resampled.values,
    text=class_counts_resampled.values,
    textposition='auto',
    marker_color='skyblue'
))

# Update layout
fig.update_layout(
    title='Distribution of Unique Classes after Oversampling',
    xaxis_title='Class',
    yaxis_title='Count'
)

# Show plot
fig.show()